In [19]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import transforms
from torch.utils.data import DataLoader

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.sub2=nn.AvgPool2d(kernel_size=2,stride=2)
        self.conv3=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        self.sub4=nn.AvgPool2d(kernel_size=2,stride=2)
        self.conv5=nn.Conv2d(in_channels=16,out_channels=120,kernel_size=4)
        self.fc6=nn.Linear(120,84)
        self.fc7=nn.Linear(84,10)
        self.tanh=nn.Tanh()

    def forward(self,x):
        x=self.conv1(x)
        x=self.tanh(x)
        x=self.sub2(x)


        x=self.conv3(x)
        x=self.tanh(x)
        x=self.sub4(x)


        x=self.conv5(x)
        x=self.tanh(x)

        x = torch.flatten(x, start_dim=1)

        x=self.fc6(x)
        x=self.tanh(x)
        x=self.fc7(x)

        return x

if __name__=='__main__':
    transform=transforms.ToTensor()
    train=datasets.MNIST(root='./data',train=True,download=True,transform=transform)
    test=datasets.MNIST(root='./data',train=False,download=True,transform=transform)
    train_dataloader=DataLoader(train,
               batch_size=4,
               shuffle=True,)
    test_dataloader=DataLoader(test,
               batch_size=4,
               shuffle=True)

    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model=LeNet5().to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
    loss=torch.nn.CrossEntropyLoss()

    loss_score=0
    best_loss = 100
    for epoch in range(200):
        for i,(data,label) in enumerate(train_dataloader):
            data=data.to(device)
            label=label.to(device)
            optimizer.zero_grad()
            output=model(data)
            loss_val=loss(output,label)
            loss_val.backward()
            optimizer.step()

            if i%1000==0:
                print('Epoch:',epoch,'Step:',i,'Loss:',loss_val.item())

        model.eval()
        eval_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # 평가에서는 그래디언트 계산을 하지 않음
            for data, labels in test_dataloader:
                data=data.to(device)
                labels=labels.to(device)
                outputs = model(data)
                loss_test = loss(outputs, labels)
                eval_loss += loss_test.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print('Accuracy:',correct/len(test))
            test_loss=eval_loss/len(test_dataloader)
            print('Test Loss:',test_loss)
            if best_loss>test_loss:
              best_loss=test_loss
              torch.save(model.state_dict(), './model_best.pth')


Epoch: 0 Step: 0 Loss: 2.344553232192993
Epoch: 0 Step: 1000 Loss: 0.019925987347960472
Epoch: 0 Step: 2000 Loss: 0.05456148087978363
Epoch: 0 Step: 3000 Loss: 0.05925726890563965
Epoch: 0 Step: 4000 Loss: 0.07700435817241669
Epoch: 0 Step: 5000 Loss: 0.054198257625103
Epoch: 0 Step: 6000 Loss: 0.03575216233730316
Epoch: 0 Step: 7000 Loss: 0.09703902900218964
Epoch: 0 Step: 8000 Loss: 0.17704366147518158
Epoch: 0 Step: 9000 Loss: 0.022009745240211487
Epoch: 0 Step: 10000 Loss: 1.1845033168792725
Epoch: 0 Step: 11000 Loss: 0.005539680831134319
Epoch: 0 Step: 12000 Loss: 0.09148440510034561
Epoch: 0 Step: 13000 Loss: 0.39787980914115906
Epoch: 0 Step: 14000 Loss: 0.013335236348211765
Accuracy: 0.9773
Test Loss: 0.07461589605547488
Epoch: 1 Step: 0 Loss: 0.006511128507554531
Epoch: 1 Step: 1000 Loss: 0.0029558632522821426
Epoch: 1 Step: 2000 Loss: 0.004235273692756891
Epoch: 1 Step: 3000 Loss: 0.017132079228758812
Epoch: 1 Step: 4000 Loss: 0.005290993023663759
Epoch: 1 Step: 5000 Loss: 0.

In [ ]:
 model=LeNet5().to(device)
 model.load_state_dict(torch.load('./model_best.pth'))
 model.eval()

 with torch.no_grad():  # 평가에서는 그래디언트 계산을 하지 않음
    for data, labels in test_dataloader:
        data=data.to(device)
        labels=labels.to(device)
        outputs = model(data)
        loss_test = loss(outputs, labels)
        eval_loss += loss_test.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy:',correct/len(test))
    test_loss=eval_loss/len(test_dataloader)
    print('Test Loss:',test_loss)